In [1]:
from pydoc import visiblename
from PIL import Image
import torch
import torchvision
from torch.utils.data.dataset import Dataset
import torchvision.transforms as transforms
import torchvision.datasets.cifar as cifar
import pickle
import os
import numpy as np
from skimage import io


In [34]:
def unpickle(file):
    with open(file, "rb") as fo:
        dic = pickle.load(fo, encoding="bytes")
    return dic


In [35]:
dic = unpickle("./data/CIFAR-10/cifar-10-batches-py/data_batch_1")


In [36]:
dic.keys()


dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

In [37]:
label_start = 0
img_root = "./data/CIFAR-10/processed/"
label = np.arange(0, 10)
for la in label:
    if not os.path.exists(f"./data/CIFAR-10/processed/train/{la}/"):
        os.makedirs(f"./data/CIFAR-10/processed/train/{la}/")
    if not os.path.exists(f"./data/CIFAR-10/processed/test/{la}/"):
        os.makedirs(f"./data/CIFAR-10/processed/test/{la}/")


In [49]:
for i in range(5):
    order = i * 10000
    dic = unpickle(f"./data/CIFAR-10/cifar-10-batches-py/data_batch_{i+1}")
    img_contain = dic[b'data']
    img_name = dic[b'filenames']
    img_label = dic[b'labels']
    for j, (label, name, data) in enumerate(zip(img_label, img_name, img_contain)):
        name = str(name).replace("b'", '').replace(".png'", '')
        img_path = f"./data/CIFAR-10/processed/train/{label}/{name}.jpg"
        io.imsave(img_path, data.reshape(
            [32, 32, 3], order="F").swapaxes(0, 1))


/tmp/ipykernel_231/2283250547.py:10: UserWarning: ./data/CIFAR-10/processed/train/2/flying_bird_s_001061.jpg is a low contrast image
  io.imsave(img_path,data.reshape([32,32,3],order="F").swapaxes(0,1))


In [ ]:
dic = unpickle(f"./data/CIFAR-10/cifar-10-batches-py/test_batch")
img_contain = dic[b'data']
img_name = dic[b'filenames']
img_label = dic[b'labels']
for j, (label, name, data) in enumerate(zip(img_label, img_name, img_contain)):
    name = str(name).replace("b'", '').replace(".png'", '')
    img_path = f"./data/CIFAR-10/processed/test/{label}/{name}.jpg"
    io.imsave(img_path, data.reshape([32, 32, 3], order="F").swapaxes(0, 1))


In [2]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [3]:
from asyncio.unix_events import BaseChildWatcher
from random import shuffle

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

train_root = r"./data/CIFAR-10/processed/train/"
test_root = r"./data/CIFAR-10/processed/test/"

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = train_transform


In [4]:
# leNet
from pyclbr import Function
import torch.nn as nn
from math import sqrt


class leNet(nn.Module):
    def __init__(self):
        super(leNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.linear1 = nn.Linear(16*5*5, 120)
        self.linear2 = nn.Linear(120, 84)
        self.linear3 = nn.Linear(84, 10)
        self.relu1 = nn.ReLU(inplace=True)
        self.relu2 = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = x.view(x.size()[0], -1)

        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)

        return x


class alexNet(nn.Module):
    def __init__(self):
        super(alexNet, self).__init__()
        self.feature = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256*2*2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 10),
        )

    def forward(self, x):
        x = self.feature(x)
        x = x.view(x.size(0), 256 * 2 * 2)
        x = self.classifier(x)
        return x


class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.first = nn.Sequential(
            nn.Conv2d(3, 18, kernel_size=3, padding=1, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        )
        self.second = nn.Sequential(
            nn.Linear(18*16*16, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 10),
        )

    def forward(self, x):
        x = self.first(x)
        x = x.view(-1, 18*16*16)
        x = self.second(x)
        return x


class VGGNet(nn.Module):
    def __init__(self, mode, batch_norm=False):
        super(VGGNet, self).__init__()
        self.features = self.make_layers(mode, batch_norm)
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 10),
        )
        # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, sqrt(2. / n))
                m.bias.data.zero_()

    def make_layers(self, mode, batch_norm):
        cfg = {
            'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
            'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
            'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
            'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M',
                  512, 512, 512, 512, 'M'],
        }
        cfg = cfg[mode]
        layers = []
        in_channels = 3
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
                if batch_norm:
                    layers += [conv2d,
                               nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
                else:
                    layers += [conv2d, nn.ReLU(inplace=True)]
                in_channels = v
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

class NiNNet(nn.Module):
    def __init__(self):
        super(NiNNet, self).__init__()
        self.classifier = nn.Sequential(
            # MLP卷积层1
            nn.Conv2d(3, 192, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 160, kernel_size=1, stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.Conv2d(160, 96, kernel_size=1, stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.Dropout(0.5),

            # MLP卷积层2
            nn.Conv2d(96, 192, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=3, stride=2, padding=1),
            nn.Dropout(0.5),

            # MLP卷积层3
            nn.Conv2d(192, 192, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=1, stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 10, kernel_size=1, stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=8, stride=1, padding=0),

        )

    def forward(self, x):
        x = self.classifier(x)
        x = nn.functional.avg_pool2d(x, kernel_size=x.size()[2:])
        x = x.view(x.size(0), 10)
        return x


In [5]:
from torch import optim


def createlossandoptimizer(net, learning_rate=0.001):
    loss = torch.nn.CrossEntropyLoss()  # 交叉熵损失函数
    # Adam 优化算法是随机梯度下降算法的扩展式
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    print(optimizer)
    return loss, optimizer


In [6]:
from torch.autograd import Variable
import time


def trainNet(net, batchsize, n_epochs, learning_rate=0.001):
    net.train()
    print("HYPERPARAMETERS: ")
    print("batch-size= ", batchsize)
    print("n_epochs= ", n_epochs)
    print("learning_rate= ", learning_rate)

    train_dataset = torchvision.datasets.ImageFolder(
        root=train_root, transform=train_transform)

    train_loader = DataLoader(train_dataset, batch_size=batchsize,
                              shuffle=True, num_workers=2)

    n_batches = len(train_loader)  # dataset size / batch size
    print("n_batches= ", n_batches)

    loss, optimizer = createlossandoptimizer(net, learning_rate)

    test_dataset = torchvision.datasets.ImageFolder(
        root=test_root, transform=test_transform)

    test_loader = DataLoader(test_dataset, batch_size=4,
                             shuffle=True, num_workers=2)

    training_start_time = time.time()

    print("training start:")
    for epoch in range(n_epochs):
        running_loss = 0.0
        print_every = n_batches
        start_time = time.time()
        total_train_loss = 0
        train_acc = 0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
# 将所有的梯度置零，原因是防止每次backward的时候梯度会累加
            optimizer.zero_grad()
            # forward
            outputs = net(inputs)
            _, pred = outputs.max(1)
            num_correct = (pred == labels).sum().item()
            acc = num_correct/batchsize
            train_acc += acc
            # loss
            loss_size = loss(outputs, labels)
            # backward
            loss_size.backward()
            # update weights
            optimizer.step()
            # print(loss_size)
            running_loss += loss_size.item()
            # 在一个epoch里。每十组batchsize大小的数据输出一次结果，即以batch_size大小的数据为一组，到第10组，20组，30组...的时候输出
            if (i + 1) % 100 == 0:
                print(
                    f"[Info] ====== {(i + 1)//batchsize*20}% ---- Loss {loss_size.item()/batchsize:.4f} ---- Accuracy {acc:.2f} ======")
        print(
            f"[Info] ====== Training ---- Epoch {epoch+1} ---- Average loss {running_loss/n_batches:.4f} ---- Accuracy {train_acc/len(train_loader):.4f} ======")
        total_val_loss = 0

# 所有的Epoch结束，也就是训练结束，计算花费的时间
    print("Training finished, took {:.2f}s".format(
        time.time() - training_start_time))


In [13]:
device = torch.device('cuda:0')

net = NiNNet().to(device)

trainNet(net, 100, 10)


HYPERPARAMETERS: 
batch-size=  100
n_epochs=  10
learning_rate=  0.001
n_batches=  500
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)
training start:
[Info] ====== 20% ---- Loss 0.0228 ---- Accuracy 0.14 ======
[Info] ====== 40% ---- Loss 0.0227 ---- Accuracy 0.21 ======
[Info] ====== 60% ---- Loss 0.0217 ---- Accuracy 0.23 ======
[Info] ====== 80% ---- Loss 0.0205 ---- Accuracy 0.30 ======
[Info] ====== 100% ---- Loss 0.0208 ---- Accuracy 0.32 ======
[Info] ====== Training ---- Epoch 1 ---- Average loss 2.1786 ---- Accuracy 0.2348 ======
[Info] ====== 20% ---- Loss 0.0209 ---- Accuracy 0.33 ======
[Info] ====== 40% ---- Loss 0.0201 ---- Accuracy 0.36 ======
[Info] ====== 60% ---- Loss 0.0206 ---- Accuracy 0.31 ======
[Info] ====== 80% ---- Loss 0.0189 ---- Accuracy 0.35 ======
[Info] ====== 100% ---- Loss 0.0188 ---- Accuracy 0.38 ======
[Info] ====== Tra